# Build kinetic models from the tfa model and sampling

In [1]:
# Load the tfa model 
from pytfa.io.json import load_json_model
# Redox corrected model
model_file = 'reduced_model_ETC_core_20250228-213124_continuous.json'
tmodel = load_json_model(model_file)
sol = tmodel.optimize()

2025-07-31 16:21:06,295 - thermomodel_Recon3thermoCurated - continuous - INFO - # Model initialized with units kcal/mol and temperature 298.15 K


In [2]:
# Import the a kinetic model 
from skimpy.io.generate_from_pytfa import FromPyTFA

small_molecules = ['h_c', 'h_e','h_i', 'h_m', 'na1_e', 'na1_c', 'na1_m','k_e','k_c']

model_gen = FromPyTFA(small_molecules=small_molecules, max_revesible_deltag_0=1e3,)

kmodel = model_gen.import_model(tmodel, sol.raw)

kmodel.name = "muscle_kinetic_model"


2025-07-31 16:21:06,831 - Unnamed - WARNING - Metabolite bhb_e in pyTFA model is not part of any reaction and is omitted in the SKiMpy model 
2025-07-31 16:21:06,832 - Unnamed - WARNING - Metabolite hdca_e in pyTFA model is not part of any reaction and is omitted in the SKiMpy model 
2025-07-31 16:21:06,833 - Unnamed - WARNING - Metabolite co2_e in pyTFA model is not part of any reaction and is omitted in the SKiMpy model 
2025-07-31 16:21:06,833 - Unnamed - WARNING - Metabolite h_e in pyTFA model is not part of any reaction and is omitted in the SKiMpy model 
2025-07-31 16:21:06,834 - Unnamed - WARNING - Metabolite lac_L_e in pyTFA model is not part of any reaction and is omitted in the SKiMpy model 
2025-07-31 16:21:06,835 - Unnamed - WARNING - Metabolite o2_e in pyTFA model is not part of any reaction and is omitted in the SKiMpy model 
2025-07-31 16:21:06,835 - Unnamed - WARNING - Metabolite glc_D_e in pyTFA model is not part of any reaction and is omitted in the SKiMpy model 


In [3]:
# Impose boundary conditions

from skimpy.core.modifiers import ConstantConcentration

for r in tmodel.boundary:
    met = [m for m in r.metabolites][0]
    try:
        k_met = kmodel.reactants[met.id]
        boundary = ConstantConcentration(k_met)
        kmodel.add_boundary_condition(boundary)
        print(k_met)
    except:
        print(f'met {met} not in model')

# Also assume intracellular sodium to be constant
# boundary = ConstantConcentration(kmodel.reactants['na1_c']) 
# kmodel.add_boundary_condition(boundary)

bhb_e
hdca_e
co2_e
h_e
met h2o_e not in model
lac_L_e
o2_e
glc_D_e


In [4]:
# Export the model to yaml
from skimpy.io.yaml import export_to_yaml
export_to_yaml(kmodel, model_file.replace("_continuous.json", "_kinetic.yml") )


True